# Imports

In [ ]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

import wandb
from wandb.keras import WandbCallback

import config
import utils
import music_model

# Training function

In [2]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATASET_NAME = 'tf_data7dict'
USE_SMALL_GENRE_SET = DATASET_NAME == 'tf_data7dict'

def get_model_name(use_mask: bool, use_reg: bool, use_mse_for_velocity:bool):
    if use_mask and not use_reg:
        model_name = 'mask_only'
    elif use_reg and not use_mask:
        model_name = 'reg_only'
    elif use_reg and use_mask:
        model_name = 'reg_and_mask'
    else:
        model_name = 'baseline'
    if use_mse_for_velocity:
        model_name += '_with_mse_vel'
    return model_name

def run_training(additional_model_name='', log_name='', model_type='GPT', 
                 use_mask=True, use_reg=True, use_mse_for_velocity=True,
                 use_wandb=True, use_one_gpu=True, verbose=True):
    
    if verbose: print("Running setup...")
    model_name = get_model_name(use_mask, use_reg, use_mse_for_velocity)
    
    if verbose: print("\tInstantiating and personalizing Config object...")
    conf = config.Config(config_string="single_instruments_type", 
                         root_path=ROOT_PATH, 
                         model_type=model_type,
                         model_name=f'model_{model_type}_{model_name + additional_model_name}')

    # Config object has by default the full list of accepted subgenres and works on multi-gpus
    # If we use the small dataset
    if USE_SMALL_GENRE_SET:
        conf.accepted_subgenres = ['folk', 'nes', 'maestro']

    # If we need to use only the first GPU
    if use_one_gpu:
        conf.GPUS = tf.config.experimental.list_physical_devices('GPU')[0]
        conf.BATCH_SIZE = 4
        conf.GLOBAL_BATCH_SIZE = conf.BATCH_SIZE
        conf.num_devices = 1
    
    if verbose: print("\tCreating model...")
    if conf.num_devices > 1:
        print("Using multiple GPUs with Mirrored Strategy")
        with conf.training_strategy.scope():
            model = music_model.create_model(conf, 
                                             use_masking_layers=use_mask,
                                             use_regularization=use_reg,
                                             use_mse_for_velocity=use_mse_for_velocity)
    else:
        print("Using single GPU/CPU device")
        model = music_model.create_model(conf, 
                                         use_masking_layers=use_mask,
                                         use_regularization=use_reg,
                                         use_mse_for_velocity=use_mse_for_velocity)
    
    if verbose: print("\tSetupping Wandb logger...")
    if use_wandb:
        wandb_config = {
            'gpus': conf.num_devices,
            'dataset': DATASET_NAME,
            'genres': conf.accepted_subgenres,
            'embedding_size': conf.SINGLE_EMB_SIZE,
            'batch_size': conf.BATCH_SIZE,
            'global_batch_size': conf.GLOBAL_BATCH_SIZE,
            'mse_for_velocity': use_mse_for_velocity,
            'reg_loss_scale': conf.REG_LOSS_SCALE,
            'masking': conf.USE_MASKING,
            'dropout_prob': conf.DROPOUT_VALUE,
            'seq_len': conf.SEQ_LEN,
            'token_dim': conf.TOKEN_DIM,
            'genre_dim': conf.GENRE_DIM,
            'attn_heads': conf.ATTENTION_HEADS,
            'attn_blocks': conf.ATTENTION_BLOCKS,
        }

        if model_type == 'GPT':
            wandb_config['activation_func'] = conf.DECODER_ACTIVATION_FUNCTION
        elif model_type == 'XL':
            wandb_config['sequence_blocks'] = conf.DIV_VAL
            wandb_config['head_dim']  = conf.HEAD_DIM
            wandb_config['inner_dim'] = conf.INNER_DIM
            wandb_config['memory_length'] = conf.MEMORY_LEN

        run = wandb.init(project="Music Generation", entity="marcello-e-federico",
                         group=model_name, job_type='train', config=wandb_config,
                         name=log_name if log_name != '' else None)
    
    if verbose: print("\tObtaining dataset...")
    dataset_path = conf.dataset_paths[DATASET_NAME]
    train_dataset, val_dataset, test_dataset = utils.get_dataset_splits(dataset_path, conf)
    
    if verbose: print("\tSetupping callbacks...")
    callbacks = conf.MODEL_CALLBACKS
    if use_wandb:
        callbacks.append(WandbCallback(
            save_model=False, save_graph=False,
            log_weights=True
        ))
    
    if verbose: print("Training start")
    history = model.fit(
        train_dataset,
        epochs = 100,
        callbacks = callbacks,
        validation_data = val_dataset,
        # initial_epoch = initial_epoch # change if resuming from previous checkpoint
    )
    if verbose: print("Training finished")
    
    if use_wandb:
        if verbose: print("\tClosing Wandb logger...")
        run.finish()
        
    if verbose: print("\tClearing session...")
    K.clear_session()
        
    return model, history

# Trainings

In [ ]:
print("TRAINING BASELINE MODEL\n\n")
model_baseline_mse_vel, history_baseline_mse_vel = run_training(additional_model_name='', log_name='', model_type='GPT', 
                                                                use_mask=False, use_reg=False, use_mse_for_velocity=True,
                                                                use_wandb=True, use_one_gpu=True, verbose=True)
print("=================================\n")
print("TRAINING MASK AND REG MODEL\n\n")
model_mask_reg_mse_vel, history_mask_reg_mse_vel = run_training(additional_model_name='', log_name='', model_type='GPT', 
                                                                use_mask=True, use_reg=True, use_mse_for_velocity=True,
                                                                use_wandb=True, use_one_gpu=True, verbose=True)
print("=================================\n")
print("TRAINING MASK ONLY MODEL\n\n")
model_mask_mse_vel, history_mask_mse_vel = run_training(additional_model_name='', log_name='', model_type='GPT', 
                                                        use_mask=True, use_reg=False, use_mse_for_velocity=True,
                                                        use_wandb=True, use_one_gpu=True, verbose=True)
print("=================================\n")
print("TRAINING REG ONLY MODEL\n\n")    
model_reg_mse_vel, history_reg_mse_vel = run_training(additional_model_name='', log_name='', model_type='GPT', 
                                                        use_mask=False, use_reg=True, use_mse_for_velocity=True,
                                                        use_wandb=True, use_one_gpu=True, verbose=True)

TRAINING BASELINE MODEL


Running setup...
	Instantiating and personalizing Config object...
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2023-01-05 16:47:32.246099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 16:47:33.399983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30503 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-01-05 16:47:33.400564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 646 MB memory:  -> device: 1, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


	Creating model...
Using single GPU/CPU device


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


	Setupping Wandb logger...


wandb: Currently logged in as: volpepe (marcello-e-federico). Use `wandb login --relogin` to force relogin


	Obtaining dataset...
	Setupping callbacks...
Training start
Epoch 1/100
1394/1394 [==============================] - 1712s 1s/step - loss: 2.2530 - type_loss: 0.0136 - measure_loss: 0.4262 - beat_loss: 0.1783 - position_loss: 0.0244 - duration_loss: 0.1537 - pitch_loss: 1.0220 - instrument_loss: 0.3388 - velocity_loss: 1.4290e-04 - key_sign_loss: 0.0437 - time_sign_loss: 0.0429 - tempo_loss: 0.0092 - val_loss: 3.2336 - val_type_loss: 0.0047 - val_measure_loss: 0.7850 - val_beat_loss: 0.2281 - val_position_loss: 0.0845 - val_duration_loss: 0.4821 - val_pitch_loss: 0.8823 - val_instrument_loss: 0.3535 - val_velocity_loss: 7.3254e-05 - val_key_sign_loss: 0.2494 - val_time_sign_loss: 0.1599 - val_tempo_loss: 0.0041 - lr: 1.0000e-04
Epoch 2/100
  14/1394 [..............................] - ETA: 25:45 - loss: 1.7026 - type_loss: 0.0071 - measure_loss: 0.0941 - beat_loss: 0.0938 - position_loss: 0.0083 - duration_loss: 0.1440 - pitch_loss: 1.0056 - instrument_loss: 0.3372 - velocity_loss: 7.5